In [83]:
"""
Veri seti yüklendi, sınıflar ve özelliklerin başlıkları güncellendi.
"""

import pandas as pd
df = pd.read_csv("../data/drug_consumption.data", header=None)
new_columns = [
    "ID", "Age", "Gender", "Education", "Country", "Ethnicity",
    "Nscore", "Escore", "Oscore", "Ascore", "Cscore",
    "Impulsiveness", "SS", "Alcohol", "Amphetamines", "Amyl_nitrite",
    "Benzodiazepines", "Cannabis", "Chocolate", "Cocaine", "Caffeine",
    "Crack", "Ecstasy", "Heroin", "Ketamine", "Legal_highs", "LSD",
    "Methadone", "Mushrooms", "Nicotine", "VSA", "Semeron"
]

df.columns = new_columns

In [84]:
"""
Madde kullanım süreleri, numerik hale getirildi.

0 (Veri Setinde: CL0): Hiç Kullanmamış
1 (Veri Setinde: CL1): 10+ Yıl Önce Kullanmış
2 (Veri Setinde: CL2): Son 10 Yıl İçinde Kullanmış
3 (Veri Setinde: CL3): Son 1 Yıl İçinde Kullanmış
4 (Veri Setinde: CL4): Son 1 Ay İçinde Kullanmış
5 (Veri Setinde: CL5): Son 1 Hafta İçinde Kullanmış
6 (Veri Setinde: CL6): Son 24 Saat İçinde Kullanmış

"""

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in df.columns:
    if df[col].dtype == 'object' and col not in ["ID", "Cocaine_binary"]:
        df[col] = le.fit_transform(df[col])


In [85]:
"""

Kokain, araştırmaya konu olan uyuşturucu olarak seçildi.
Kokain Kullanım Durumunu gösteren bir sınıf oluşturuldu (binary class).
[0,1]: Kullanmıyor - 0,
[2,6]: Kullanıyor - 1.

"""

df["Cocaine_binary"] = df["Cocaine"].apply(
    lambda x: 1 if x >= 2 else 0
)

In [86]:
"""

Özellik seçimi ve modellerin eğitimi için gereken train-test ayrımı yapıldı.
X: Bağımsız Değişken(ler)
Y: Bağımlı Değişken 

"""

from sklearn.model_selection import train_test_split

X = df.drop(["Cocaine", "Cocaine_binary"], axis=1)
y = df["Cocaine_binary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [87]:
"""

Feature Selection 1: Information Gain ile En Değerli Özellikleri Belirleme

Entropi ve Information Gain hesapları yapıldı.
En düşük gain'i 20 feature listelendi.


"""

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(X, y, random_state=42)
mi_df = pd.DataFrame({
    "feature": X.columns,
    "mi_score": mi_scores
}).sort_values(by="mi_score", ascending=False)

print("En az Information Gain sağlayan 20 özellik: ")
mi_df.tail(20)

En az Information Gain sağlayan 20 özellik: 


,feature,mi_score
3,Education,0.001458
26,Methadone,0.001455
6,Nscore,0.001208
29,VSA,0.000852
15,Amyl_nitrite,0.000517
27,Mushrooms,0.000000
23,Ketamine,0.000000
24,Legal_highs,0.000000
22,Heroin,0.000000
0,ID,0.000000


In [88]:
"""

Feature Selection 2: ReliefF ile En İyi Ayrım Yaptırmaya Yarayan Özellikleri Belirleme

En az ağırlığa sahip olan 20 feature listelendi.

"""

from skrebate import ReliefF

relief = ReliefF(n_neighbors=100)
relief.fit(X_train.values, y_train.values)

relief_scores = relief.feature_importances_

relief_df = pd.DataFrame({
    "feature": X.columns,
    "relief_score": relief_scores
}).sort_values(by="relief_score", ascending=False)

print("ReliefF'e göre sınıf ayırmada en başarısız 20 özellik: ")
relief_df.tail(20)

ReliefF'e göre sınıf ayırmada en başarısız 20 özellik: 


,feature,relief_score
20,Crack,0.227740
3,Education,0.225754
13,Alcohol,0.216382
24,Legal_highs,0.216360
21,Ecstasy,0.212613
26,Methadone,0.198842
22,Heroin,0.198160
5,Ethnicity,0.189911
15,Amyl_nitrite,0.182385
30,Semeron,0.166799


In [89]:
""" 
Min-Max Normalizasyonu ile değerleri 0-1 arasına çekelim: Ki-Kare Testi için gerekli.
"""

from sklearn.preprocessing import MinMaxScaler

scaler_for_chisq = MinMaxScaler()
X_scaled_fs = scaler_for_chisq.fit_transform(X)

In [90]:
"""

Feature Selection 3: Ki-Kare Testi.

"""

from sklearn.feature_selection import SelectKBest, chi2

chi2_selector = SelectKBest(score_func=chi2, k='all')
chi2_selector.fit(X_scaled_fs, y)
chi2_scores = chi2_selector.scores_

In [91]:
"""

Feature Selection 3: Ki-Kare Testi.

Feature'lar ile hedef değişken arasındaki ilişkilerin gösterimi gerçekleştiriliyor.
En zayıf ilişkiye sahip 20 feature listelendi.

"""

chi2_df = pd.DataFrame({
    "feature": X.columns,
    "chi2_score": chi2_scores
}).sort_values(by="chi2_score", ascending=False)

print("Ki-Kare yöntemine göre sınıf ayırmada en başarısız kalan 20 özellik:")
chi2_df.tail(20)

Ki-Kare yöntemine göre sınıf ayırmada en başarısız kalan 20 özellik:


,feature,chi2_score
1,Age,0.468094
0,ID,0.376998
15,Amyl_nitrite,0.311435
27,Mushrooms,0.299053
30,Semeron,0.261184
28,Nicotine,0.218980
16,Benzodiazepines,0.197157
3,Education,0.184572
26,Methadone,0.150993
8,Oscore,0.133712


In [92]:
"""
Modellerin çalışması için gerekli olan algoritmalar ve
yorumlamalar için gereken metrikler ithal ediliyor
"""

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [93]:
"""
Lojistik Regresyon ve Destek Vektörü Makinesi Algoritmaları 
için gerekli olan ölçeklenmiş veriler hazırlanıyor
"""
scaler_model = StandardScaler()
X_train_scaled = scaler_model.fit_transform(X_train)
X_test_scaled = scaler_model.transform(X_test)

In [94]:
"""
Sonuçları ekrana yazdıracak fonksiyon
"""

def bilgi_yazdir(name, y_true, y_pred):
    print(f"\n{name} Sonuçları\n")
    print("--------------------------")
    print(f"Accuracy Skoru: %{accuracy_score(y_true, y_pred) * 100:.2f} ")
    print(f"Precision Skoru: %{precision_score(y_true, y_pred) * 100:.2f}")
    print(f"Recall Skoru: %{recall_score(y_true, y_pred) * 100:.2f}")
    print(f"F1 Skoru: %{f1_score(y_true, y_pred) * 100:.2f}")
    print("Confusion Matrisi:\n", confusion_matrix(y_true, y_pred))
    print("--------------------------")

In [95]:
"""
Lojistik Regresyon Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""

lojistik_regresyon = LogisticRegression(
    max_iter=500,
    class_weight="balanced", 
    random_state=42
)
lojistik_regresyon.fit(X_train_scaled, y_train)
y_tahmin_lr = lojistik_regresyon.predict(X_test_scaled)

In [96]:
"""
Destek Vektörü Makinesi Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""
destek_vektoru = SVC(
    kernel="rbf",
    class_weight="balanced",
    probability=False,
    random_state=42
)

destek_vektoru.fit(X_train_scaled, y_train)
y_tahmin_svm = destek_vektoru.predict(X_test_scaled)

In [97]:
"""
Rassal Orman Algoritması 1: Feature Selection İşlemleri Gerçekleştirilmemiş Versiyon
"""

rassal_orman = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)

rassal_orman.fit(X_train, y_train)
y_tahmin_rf = rassal_orman.predict(X_test)

In [98]:
"""
Eğitim veriseti, FS algoritmalarının belirlediği 9 adet ortak
en düşük değere sahip feature'lar silinecek şekilde düzenleniyor
"""

chi2_en_dusukler = chi2_df.sort_values(by="chi2_score").head(20)["feature"].tolist()

relief_en_dusukler = relief_df.sort_values(by="relief_score").head(20)["feature"].tolist()

mi_en_dusukler = mi_df.sort_values(by="mi_score").head(20)["feature"].tolist()

ortak_dusukler = set(chi2_en_dusukler) & set(relief_en_dusukler) & set(mi_en_dusukler)

print("FS Algoritmalarının ortak olarak gösterdiği en az değerli feature'lar: ")
for c in ortak_dusukler:
    print(f"{c}, ")

FS Algoritmalarının ortak olarak gösterdiği en az değerli feature'lar: 
SS, 
Cscore, 
Education, 
Ethnicity, 
Methadone, 
Nscore, 
Amyl_nitrite, 
Semeron, 
ID, 


In [99]:
"""
En az değerli feature'lar silinerek yeni bir eğitim seti oluşturuluyor
"""

silinecek = list(ortak_dusukler)
final_drop_list = [col for col in silinecek + ["Cocaine", "Cocaine_binary"] if col in df.columns]
X_reduced = df.drop(columns=final_drop_list)

In [100]:
"""
Yeni train_test_split gerçekleştiriliyor
"""

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reduced, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [101]:
"""
Azaltılmış veri setinin ölçeklenme işlemleri gerçekleştiriliyor
"""
scaler_r = StandardScaler()
X_train_r_scaled = scaler_r.fit_transform(X_train_r)
X_test_r_scaled = scaler_r.transform(X_test_r)

In [102]:
"""
Lojistik Regresyon Algoritması 2: Feature Selection Sonrası Versiyon
"""
lojistik_regresyon_reduced = LogisticRegression(
    max_iter=500,
    class_weight="balanced",
    random_state=42
)

lojistik_regresyon_reduced.fit(X_train_r_scaled, y_train_r)
y_tahmin_lr_reduced = lojistik_regresyon_reduced.predict(X_test_r_scaled)

In [103]:
"""
Destek Vektör Makinesi Algoritması 2: Feature Selection Sonrası Versiyon
"""

destek_vektoru_reduced = SVC(
    kernel="rbf",
    class_weight="balanced",
    random_state=42
)

destek_vektoru_reduced.fit(X_train_r_scaled, y_train_r)
y_tahmin_svm_reduced = destek_vektoru_reduced.predict(X_test_r_scaled)

In [104]:
"""
Rassal Orman Algoritması 2: Feature Selection Sonrası Versiyon
"""

rassal_orman_reduced = RandomForestClassifier(
    n_estimators=300,
    class_weight="balanced",
    random_state=42
)

rassal_orman_reduced.fit(X_train_r, y_train_r)
y_tahmin_rf_reduced = rassal_orman_reduced.predict(X_test_r)

In [105]:
""" 
Algoritmalar ve yaptıkları tahminlemeler eşleştiriliyor
"""

tahminler_fs_oncesi = {
    "Lojistik Regresyon (FS Öncesi)": y_tahmin_lr, 
    "Destek Vektör Makinesi (FS Öncesi)": y_tahmin_svm, 
    "Rassal Orman (FS Öncesi)": y_tahmin_rf 
}
tahminler_fs_sonrasi = {
    "Lojistik Regresyon (FS Sonrası)": y_tahmin_lr_reduced,
    "Destek Vektör Makinesi (FS Sonrası)": y_tahmin_svm_reduced,
    "Rassal Orman (FS Sonrası)": y_tahmin_rf_reduced
}

In [106]:
"""
Feature Selection işlemlerinden önceki tahminlemeler ve skorlar:
"""
for model_adi, tahmin in tahminler_fs_oncesi.items():
    bilgi_yazdir(model_adi, y_test, tahmin)


Lojistik Regresyon (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %77.19 
Precision Skoru: %98.30
Recall Skoru: %78.11
F1 Skoru: %87.05
Confusion Matrisi:
 [[  2   5]
 [ 81 289]]
--------------------------

Destek Vektör Makinesi (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %93.63 
Precision Skoru: %98.06
Recall Skoru: %95.41
F1 Skoru: %96.71
Confusion Matrisi:
 [[  0   7]
 [ 17 353]]
--------------------------

Rassal Orman (FS Öncesi) Sonuçları

--------------------------
Accuracy Skoru: %98.14 
Precision Skoru: %98.14
Recall Skoru: %100.00
F1 Skoru: %99.06
Confusion Matrisi:
 [[  0   7]
 [  0 370]]
--------------------------


In [107]:
"""
Feature Selection işlemlerinden sonraki tahminlemeler ve skorlar:
"""

for model_adi, tahmin in tahminler_fs_sonrasi.items():
    bilgi_yazdir(model_adi, y_test_r, tahmin)


Lojistik Regresyon (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %74.80 
Precision Skoru: %98.25
Recall Skoru: %75.68
F1 Skoru: %85.50
Confusion Matrisi:
 [[  2   5]
 [ 90 280]]
--------------------------

Destek Vektör Makinesi (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %92.84 
Precision Skoru: %98.04
Recall Skoru: %94.59
F1 Skoru: %96.29
Confusion Matrisi:
 [[  0   7]
 [ 20 350]]
--------------------------

Rassal Orman (FS Sonrası) Sonuçları

--------------------------
Accuracy Skoru: %98.14 
Precision Skoru: %98.14
Recall Skoru: %100.00
F1 Skoru: %99.06
Confusion Matrisi:
 [[  0   7]
 [  0 370]]
--------------------------


In [108]:
"""
Kokain kullanıcılarının bazı yasal ve yasa dışı uyuşturucu madde tüketimlerini inceleyelim.
Hatırlatma:
0 (Veri Setinde: CL0): Hiç Kullanmamış
1 (Veri Setinde: CL1): 10+ Yıl Önce Kullanmış
2 (Veri Setinde: CL2): Son 10 Yıl İçinde Kullanmış
3 (Veri Setinde: CL3): Son 1 Yıl İçinde Kullanmış
4 (Veri Setinde: CL4): Son 1 Ay İçinde Kullanmış
5 (Veri Setinde: CL5): Son 1 Hafta İçinde Kullanmış
6 (Veri Setinde: CL6): Son 24 Saat İçinde Kullanmış
"""

def cocaine_group(x):
    if x in [0,1,2]:
        return "Düşük Kullanıcı (CL0, CL1, CL2)"
    elif x in [3,4]:
        return "Orta Düzey Kullanıcı (CL3, CL4)"
    return "Aktif Kullanıcı (CL5, CL6)"


In [109]:
"""
Legal Uyuşturucular: Kokain ve nikotin kullanımı:
"""

nicotine_coke = pd.crosstab(
    df["Nicotine"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
nicotine_coke


Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Nicotine,,,
0,82.710280,2.102804,15.186916
1,76.683938,4.145078,19.170984
2,81.372549,1.960784,16.666667
3,77.297297,1.621622,21.081081
4,79.629630,0.925926,19.444444
5,77.707006,3.184713,19.108280
6,77.213115,2.459016,20.327869


In [110]:
"""
Kokain ve alkol kullanımı:
"""

alcohol_coke = pd.crosstab(
    df["Alcohol"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
alcohol_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Alcohol,,,
0,67.647059,14.705882,17.647059
1,79.411765,5.882353,14.705882
2,85.294118,2.941176,11.764706
3,77.777778,1.010101,21.212121
4,77.351916,2.439024,20.209059
5,79.973650,2.239789,17.786561
6,79.009901,1.980198,19.009901


In [111]:
"""
Kokain ve kafein kullanımı:
"""

caffeine_coke = pd.crosstab(
    df["Caffeine"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
)
caffeine_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Caffeine,,,
0,0.815992,0.023121,0.160886
1,0.743750,0.025000,0.231250
2,0.818519,0.029630,0.151852
3,0.728682,0.031008,0.240310
4,0.727273,0.010101,0.262626
5,0.707317,0.000000,0.292683
6,0.736842,0.000000,0.263158


In [112]:
"""
Kokain ve esrar kullanımı:
"""

cannabis_coke = pd.crosstab(
    df["Cannabis"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
cannabis_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Cannabis,,,
0,66.666667,7.407407,25.925926
1,50.000000,30.000000,20.000000
2,66.666667,4.166667,29.166667
3,63.333333,3.333333,33.333333
4,77.358491,1.886792,20.754717
5,77.289377,2.564103,20.146520
6,80.866426,2.021661,17.111913


In [113]:
amph_coke = pd.crosstab(
    df["Amphetamines"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
amph_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Amphetamines,,,
0,81.762295,2.561475,15.676230
1,80.434783,2.608696,16.956522
2,78.600823,1.646091,19.753086
3,72.222222,3.030303,24.747475
4,68.000000,2.666667,29.333333
5,80.327869,0.000000,19.672131
6,71.568627,1.960784,26.470588


In [114]:
benzo_coke = pd.crosstab(
    df["Benzodiazepines"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
benzo_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Benzodiazepines,,,
0,81.400000,2.100000,16.500000
1,79.310345,2.586207,18.103448
2,79.914530,2.991453,17.094017
3,72.881356,2.966102,24.152542
4,72.500000,0.833333,26.666667
5,78.571429,2.380952,19.047619
6,75.789474,4.210526,20.000000


In [115]:
ecstasy_coke = pd.crosstab(
    df["Ecstasy"],
    df["Cocaine"].map(cocaine_group),
    normalize="index"
) * 100
ecstasy_coke

Cocaine,"Aktif Kullanıcı (CL5, CL6)","Düşük Kullanıcı (CL0, CL1, CL2)","Orta Düzey Kullanıcı (CL3, CL4)"
Ecstasy,,,
0,81.782566,2.448580,15.768854
1,79.646018,4.424779,15.929204
2,75.213675,4.273504,20.512821
3,77.617329,1.083032,21.299639
4,73.717949,0.641026,25.641026
5,66.666667,1.587302,31.746032
6,80.952381,0.000000,19.047619
